# **Introduction**
We are going through to make a classifier for rock, paper, and scissors images using Convolutional Neural Network (CNN) with the help of TensorFlow and Keras. We also going to use Hyperparameter Tuning to help find the optimal model. Happy exploring!

# **Library**
## Import Libraries and Packages
The main library for this project are TensorFlow and its package Keras. So, the first thing you need is to import TensorFlow (make sure you already install the TensorFlow) and Keras will right away imported too. Then to create a new data from our dataset we use ImageDataGenerator from Keras for our image augmentation step.

Note: There are some libraries present in the code cells below this section because I want to show what those libraries do.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Data Preparation**
## Download and Extract The Dataset
Next, we are going to download the dataset using wget command from the link that have been provided from my learning platform you may use it as well if you run it through Google Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Extract the zip file

In [ ]:
import zipfile
# Open the zip file
with zipfile.ZipFile('/content/drive/MyDrive/Data FAR/fer2013.zip', 'r') as zip_ref:
    # Extract all the files to the current directory
    zip_ref.extractall()

Then we set the main directory of our project to load the dataset

In [ ]:
BASE_TRAIN = '/content/train/'
BASE_TEST = '/content/test'

# **Data Preprocessing**

After we prepare our dataset then we are going to preprocess our dataset. In the Training and Validation datasets, several features are used for image augmentation such as rescale, rotation, changing image shifts, zooming, flipping horizontally, and filling pixels after being changed with previous features. Then we split the dataset into 60:40 where the training is 60% and validation is 40%.

## Training Dataset

In [ ]:
training_datagen = ImageDataGenerator(rescale=1. / 255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

training_generator = training_datagen.flow_from_directory(BASE_TRAIN,
                                                          target_size=(150, 150),
                                                          batch_size=32,
                                                          class_mode='categorical',
                                                          shuffle=True,
                                                          seed=42)


Found 28709 images belonging to 7 classes.


To print out the class label and as you can see there are 3 classes which are 'paper', 'rock', and 'scissors'.

In [ ]:
# Get the class labels
training_class_labels = list(training_generator.class_indices.keys())
training_class_labels

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

## Validation Dataset

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1. / 255,
                                        rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')

validation_generator = validation_datagen.flow_from_directory(BASE_TEST,
                                                              target_size=(150, 150),
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              shuffle=False,
                                                              seed=42)


Found 7178 images belonging to 7 classes.


Same as the training dataset, validation dataset also has 3 classes which are 'paper', 'rock', and 'scissors'.

In [ ]:
# Get the class labels
validation_class_labels = list(validation_generator.class_indices.keys())
validation_class_labels

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# **Machine Learning Modelling**
Before we do modeling, we will do Hyperparameter Tuning to find the best parameters to use in our model.

## **Hyperparameter Tuning**
To do Hyperparameter Tuning first we need to install 'keras tuner' to help searching the best parameter.

First install 'keras_tuner' with pip

In [ ]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


Then import the necessary library and packages

In [ ]:
import keras_tuner as kt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

Next, we build a function to build our model using hyperparameter tuning. There are several differences to this code than the usual model building with TensorFlow. In this function, we separate the convolutional input layer, convolutional hidden layer, and pooling hidden layer. Because, in the input layer we only need to find best convolutional units by setting a minimum value of 16 and a maximum value of 256 with an increment step of 16. Meanwhile, in the hidden layer, we use a loop to determine how many layers are the best. The parameter we are going to search are convolutional units and kernel size of conv2D and for the MaxPooling2D we are going to use the same kernel size. We are also going to search best parameter for dropout layer. For the dense layer, we also use the same loop as the conv2D hidden layer and look for how many dense layer are the best while also searching the best dense units. Then, there is no change in the output layer because we only have 3 classes so the dense units are 3 with 'softmax' activation because this is a multiclass classification. Last, we are going to search the best optimizer and for the loss we only use 'categorical_crossentropy' because this is a multiclass classification.

In [ ]:
def model_builder(hp):
    model = keras.Sequential()

    # Add input convolutional layers
    model.add(keras.layers.Conv2D(hp.Int('conv1_units', min_value=16, max_value=256, step=16), (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(keras.layers.MaxPooling2D((2, 2)))

    # Add hidden layers
    for i in range(hp.Int('num_conv_layers', 1, 4)):
        model.add(keras.layers.Conv2D(hp.Int(f'conv{i+2}_units', min_value=32, max_value=512, step=32),
                                      kernel_size=hp.Int(f'conv{i+2}_kernel_size', min_value=3, max_value=5, step=2),
                                      activation='relu'))
        model.add(keras.layers.MaxPooling2D((2, 2)))

    # Flatten the output
    model.add(keras.layers.Flatten())

    # Dropout layer
    model.add(keras.layers.Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Add dense layers
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(keras.layers.Dense(units=hp.Int(f'dense{i+1}_units', min_value=32, max_value=512, step=32), activation='relu'))

    # Output layer
    model.add(keras.layers.Dense(7, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd', 'Nadam']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

After we build the model builder function, we make a tuner for our hyperparameter tuning.

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=7,
                     directory='my_dir',
                     project_name='rps_kt')

This is a callback to do early stopping by specifying the number of epochs with no improvement after which training will be stopped. In this case, if the validation accuracy does not improve for three consecutive epochs, the training will stop early.

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

We will perform a search for the best parameters, with a maximum of 10 epochs

In [ ]:
tuner.search(training_generator,
             validation_data=validation_generator,
             epochs=10,
             callbacks=[stop_early])

Epoch 1/10
898/898 [==============================] - 254s 279ms/step - loss: 1.8198 - accuracy: 0.2448 - val_loss: 1.7882 - val_accuracy: 0.2552
Epoch 2/10
898/898 [==============================] - 248s 277ms/step - loss: 1.7817 - accuracy: 0.2586 - val_loss: 1.7650 - val_accuracy: 0.2618
Epoch 3/10
898/898 [==============================] - 249s 277ms/step - loss: 1.7608 - accuracy: 0.2710 - val_loss: 1.7408 - val_accuracy: 0.2813
Epoch 4/10
898/898 [==============================] - 249s 277ms/step - loss: 1.7337 - accuracy: 0.2848 - val_loss: 1.7073 - val_accuracy: 0.2990
Epoch 5/10
898/898 [==============================] - 249s 277ms/step - loss: 1.7031 - accuracy: 0.3092 - val_loss: 1.7794 - val_accuracy: 0.3040
Epoch 6/10
 48/898 [>.............................] - ETA: 3:22 - loss: 1.6928 - accuracy: 0.3242

To print out the best trial from hyperparameter tuning

In [ ]:
# Get the best trials
best_trials = tuner.oracle.get_best_trials(num_trials=1)

In [ ]:
# Print information about the best trial
for best_trial in best_trials:
    print(f"Best trial number: {best_trial.trial_id}")
    print(f"Best trial value (objective): {best_trial.score}")
    print("Best trial hyperparameters:")
    for param, value in best_trial.hyperparameters.values.items():
        print(f"{param}: {value}")

In [ ]:
# Print the best optimizer and loss
best_optimizer = best_trial.hyperparameters.get('optimizer')
print(f"Best optimizer: {best_optimizer}")

## **Best Model Building**

We can build the model that we will use from the results of the best hyperparameter tuning here.

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final best model from best parameters
hypermodel = tuner.hypermodel.build(best_hps)

In [ ]:
# Summary of the best model
hypermodel.summary()

# **Model Re-Training**

After we did the hyperparameter tuning and build the best model, we are going to re-training. Several features are implemented such as measuring the length of model training with the 'Time' library and using a callback feature to stop the training process if validation accuracy has reached above 96%.

In [ ]:
import time
# Start the timer
start_time = time.time()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('val_accuracy') > 0.96):
      print("\nReached 96% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

history = hypermodel.fit(training_generator,
                         epochs=15,
                         steps_per_epoch=32,
                         validation_data=validation_generator,
                         callbacks=callbacks)

# Stop the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = round(end_time - start_time) / 60
print(f"Elapsed time: {elapsed_time} minutes")

We achieved validation accuracy of 96.34% for 3.43 minutes!

# **Model Evaluation**

Model evaluation from the results of the training process is shown by creating accuracy plots and loss plots for training and validation.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Plot Accuracy on the first subplot
ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Model Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

# Plot Loss on the second subplot
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Model Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()

plt.show()

# **Model Test By Using Image Upload**

Next, we are going to test the model by uploading images of rock, paper, scissors to carry out image detection. Also added are the prediction results and probability values which are presented in the form of a bar plot.

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

uploaded = files.upload()

class_labels = {0: 'paper', 1: 'rock', 2: 'scissors'}

# Set up the subplots
fig, ax = plt.subplots(nrows=len(uploaded), ncols=2, figsize=(10, len(uploaded)*5))

for i, fn in enumerate(uploaded.keys()):
  # predict images
  path = fn
  img_source = image.load_img(path, target_size = (150, 150))
  x = image.img_to_array(img_source)
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  classes = hypermodel.predict(images, batch_size = 10)

  print(fn)

  # Use argmax to find the index of the predicted class
  predicted_class_index = np.argmax(classes[0])

  # Use the class_labels dictionary to get the corresponding label
  predicted_class = class_labels[predicted_class_index]

  print(predicted_class)

  # Display the image on the left side
  ax[i, 0].imshow(img_source)
  ax[i, 0].axis('off')  # Turn off axis labels

  # Bar plot of class probabilities on the right side
  class_probabilities = classes[0]
  ax[i, 1].barh(list(class_labels.values()), class_probabilities)
  ax[i, 1].set_xlim([0, 1])  # Set y-axis limit to match probability range
  ax[i, 1].set_xlabel('Probability')
  ax[i, 1].set_title('Class Probabilities')

  # Print the predicted class label
  ax[i, 1].text(1.1, 0.5, f'Predicted: {predicted_class}', transform=ax[i, 1].transAxes, fontsize=12,
                verticalalignment='center')

plt.show()

# **Conclusion**
Hyperparameter Tuning proves beneficial if we are uncertain regarding the selection of parameters and optimal number of layers for our model. While this may help to do 'semi-automation' in the search for the best configuration. However, it is important to note that hyperparameter tuning can be computationally expensive. As you can see, it takes quite some time to building the tuner, searching the best parameter, and re-training so it might takes some consideration to use this. A high performance PC is recommended due it will take high computational resources to train with images dataset.

The model performance is quite good in detecting images with green background but it fails to detect images with different background so it may need another dataset to improve the accuracy for unseen data.